In [1]:
import numpy as np
f = open('resources/InsectWingbeatSound/InsectWingbeatSound_TEST','r')
data = f.read()
f.close()
# 개행문자 기준으로 끊어서 리스트로
data_list = data.split('\n')

# "," 기준으로 끊어서 리스트로
emptylist = []
for list_part in data_list:
    emptylist.append(list_part.split(","))

In [2]:
# str -> float 변환
tofloat = []
for partlist in emptylist:
    tofloat.append([float(i) for i in partlist]) 

In [3]:
labels = []
data_list = []
for datas in tofloat:
    labels.append(datas[0])
    data_list.append(datas[1:])
print(np.shape(labels))
print(np.shape(data_list))


(1980,)
(1980, 256)


In [4]:
from readFile import split_into_values, toRPdata
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler

def Standard(data):
    SS = StandardScaler().fit(data)
    scaled = SS.transform(data)
    return scaled

def MinMax(data):
    MMS = MinMaxScaler().fit(data)
    scaled = MMS.transform(data)
    return scaled

# result_list transpose
result_T = [list(x) for x in zip(*data_list)]

# minmax 정규화
result_scaled = Standard(result_T)

# 다시 result transpose 해서 원래대로
result_scaled = [list(x) for x in zip(*result_scaled)]

result_ = np.array(result_scaled)

In [5]:
result_.shape

(1980, 256)

In [6]:
data = result_.reshape(result_.shape[0], 1, result_.shape[1])
X = toRPdata(data, threshold='point', percentage=30)
#X = toRPdata(data)
    
X_scaled = np.expand_dims(X, axis=3)
X_scaled.shape

(1980, 256, 256, 1)

In [20]:
from utils import split_data, normalization_tool
from agent import Autoencoder_Agent
from keras import backend as K
import tensorflow as tf

In [106]:
learning_rate = 3e-5
optimizer='Adam'
loss='mse'
image_size = 256 #1024, 256
dimension = 8 # ...

In [107]:
autoencoder = Autoencoder_Agent(model_size=image_size, dimension=dimension, optimizer=optimizer,learning_rate=learning_rate)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_54 (Conv2D)           (None, 256, 256, 128)     1280      
_________________________________________________________________
activation_54 (Activation)   (None, 256, 256, 128)     0         
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 128, 128, 128)     0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 128, 128, 64)      73792     
_________________________________________________________________
activation_55 (Activation)   (None, 128, 128, 64)      0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 64, 64, 32)       

In [108]:
autoencoder.load(f'model/insect_128_{dimension}_bce.h5')

In [123]:
all_feature = np.empty((0,dimension), float)
for i in range(66):
    all_feature = np.append(all_feature, autoencoder.feature_extract(X_scaled[i*30:(i+1)*30]), axis=0)

print(all_feature.shape)

(1980, 8)


In [135]:
from sklearn.cluster import KMeans
cluster_result = KMeans(n_clusters=10).fit(origin_data)
plotSilhouette(origin_data,cluster_result)

0.22493226471341543

In [89]:
# 1. 해당 dimension 에서 100번 돌려서 제일 좋은 결과를 뽑는다.
max_result = -1
for _ in range(100):
    result = KMeans(n_clusters=11).fit(features)
    res = plotSilhouette(result_,result)
    if max_result < res:
        max_result = res
print(f"군집 11 최대 실루엣 점수: {max_result}")

군집 11 최대 실루엣 점수: -0.03062825026564111


In [90]:
# 2. 1 ~ 100 군집 개수로 인한 결과 중에서 가장 좋은 결과를 뽑는다.
max_case_result = -1
max_case_cluster_n = 0
for i in range(5, 101):
    result = KMeans(n_clusters=i).fit(features)
    res = plotSilhouette(result_,result)
    if max_case_result < res:
        max_case_result = res
        max_case_cluster_n = i
print(f"최대 실루엣 점수: {max_case_result}\n군집개수: {max_case_cluster_n}")

KeyboardInterrupt: 

In [55]:
from sklearn.metrics import silhouette_samples,silhouette_score

def plotSilhouette(X, y_km):
    cluster_labels = np.unique(y_km.labels_)
    n_clusters = cluster_labels.shape[0]
    silhouette_vals = silhouette_score(X, y_km.labels_,metric='euclidean')
#     print(silhouette_vals)
    return silhouette_vals

In [95]:
origin_data = np.array(data_list)
origin_data.shape

(1980, 256)

In [125]:
# 각 label 별 리스트 나누기
# labels : 기존 라벨
# all_feature : dataset

# labels == 1 인 인덱스 : idx
# cluster_1 = all_feature[idx]

for i in range(1, 12):
#     features = np.empty((1,32))
    indexes = []
    for idx, label in enumerate(labels):
        if label == i:
            indexes.append(idx)
    features = origin_data[indexes]
    result = cluster_result.predict(features)
    print(result)
#     print(features)
#     print(features.shape)

# 리스트 별로 predict 진행
# 결과 저장

[ 4  0  4  4  4  0  0  4  0  0  4  0  0  4  0  0  4 10  4  0  0 10  0  4
  0  4  4  0  0  0  4  4  0  0  0  0  0  4  4  4  0  0  4  4  0  0  0  4
  0  0  4  0  0  4 10  0  0  0  4  0  0  0  4  0  0  4  0  4  0  4  4  4
  0  0  0  0  4  4  4  0  4  0  0 10  4  0  4  0  4 10  4  0  0  0 10  0
  0  0  0  4  4  4  4  0  0  0  0  0  0  4  4  4  0  4  4  4  4  4  4 10
  0  4  4  4  4  0  4  0  0 10  4  0  4  0  4  4  0  4  0 10  0  0  0  4
  4  0  4  0  0  4  0  0  4  0  4  0  0  4  0  4  4  0  0  0  0  4  4  4
  4  0 10  0  4  0  0  4  0  0  4  0]
[ 3  7  3  7  3  5  3  2  3  7 10  2 10  8  3  7  1  3  3  7  3  0  5  7
  3  7  3  3  8  7  2  3  7  7  3  3  3  3 10  7  7  3  3  3  3  4  3  3
  3  3  7  3  3  7  7  3  7  3  3  7  3  7  3  7  6  8  3  6  2  3  3  7
  7  7  3  2  2  2  7  7  3  3  3  3  3  9  7  6  1  3  7  3  3  3  3  6
  3  3  3  3  7  3  3  7  7  2  7  3  7  5  3  7  7  3  7  3  2  7  7  2
  3  7  7  2  5  5  2  5  7  2  3  3  7  7  3  3  8  3  1  3  3  8  7  7
 10  7  3  2 

In [131]:
labels_ = cluster_result.labels_ + 1
dic_labels_ = {}
for i in labels_:
    if f'{i}' not in dic_labels_.keys():
        dic_labels_[f'{i}'] = 1
    else:
        dic_labels_[f'{i}'] += 1

for key, value in dic_labels_.items():
    print(f"{key}: {value}")

3: 216
2: 286
9: 180
4: 145
1: 203
5: 149
10: 120
11: 209
6: 184
7: 197
8: 91


In [128]:
# 각 label 별 리스트 나누기
# labels : 기존 라벨
# all_feature : dataset

# labels == 1 인 인덱스 : idx
# cluster_1 = all_feature[idx]

for i in range(1, 12):
#     features = np.empty((1,32))
    indexes = []
    for idx, label in enumerate(labels):
        if label == i:
            indexes.append(idx)
    features = all_feature[indexes]
    result = cluster_result.predict(features)
    print(result)
#     print(features)
#     print(features.shape)

# 리스트 별로 predict 진행
# 결과 저장

[7 2 7 5 8 2 6 2 5 7 0 5 2 5 7 0 7 8 6 2 8 5 5 2 7 5 2 5 8 7 6 5 0 2 7 2 0
 2 5 7 2 7 2 5 3 2 7 5 0 2 2 2 8 5 5 5 7 5 2 6 0 8 0 5 2 8 7 7 5 0 2 5 8 7
 6 7 5 5 2 2 5 5 5 8 5 7 5 5 7 2 5 6 5 5 5 2 5 0 0 8 2 2 7 2 7 2 6 7 8 5 0
 2 5 5 2 5 5 2 8 8 0 5 7 2 2 7 6 2 2 7 5 2 2 2 2 2 2 5 7 2 2 2 2 2 2 2 5 2
 0 5 5 7 5 5 2 5 7 5 6 7 0 7 8 6 0 7 2 7 2 0 8 7 5 7 7 8 2 0 7 7]
[ 1 10  3  5  9  0  1  7  1  9 10  1  0  3  0 10  2  9  3 10 10  5  1  1
  0  5  6  3  3 10  1  1 10 10 10  4  0  1  7  0  1 10  3  7  1  0  0  4
  3  3  1  1  4 10 10  1  2  4  3  5  1  1  0 10 10  1 10 10  4  1  9  6
  1  0  6  1  1  5  5 10  0  1 10 10  0  6  6  2  5  1  3 10  1  6 10  0
  4  1  9  1  6  1  9  1 10  9 10  7  6  3 10 10  0 10 10  1  3 10 10  9
  1 10  2  6  5  5  1  3  1  1  3  3 10  1  6 10  1 10  8  1  3  3  6  6
  9  1  1 10 10  1  3 10  6  1 10  3 10  1  1  1 10 10 10 10 10  1  6  6
  9  3  6  7  1  6 10  4  3  0  1  1]
[ 0  5  5  5  5  7  5  2  5  8  7  7  7  6  7  6  5  8  6  0  6  7  6  5
  7  7  6 10

In [129]:
labels_ = cluster_result.labels_ + 1
dic_labels_ = {}
for i in labels_:
    if f'{i}' not in dic_labels_.keys():
        dic_labels_[f'{i}'] = 1
    else:
        dic_labels_[f'{i}'] += 1

for key, value in dic_labels_.items():
    print(f"{key}: {value}")

1: 163
6: 384
7: 198
8: 241
4: 120
11: 144
2: 112
3: 248
5: 63
9: 271
10: 36
